批量处理

1. try some tokenizer

In [6]:
import pandas as pd
import re
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
def tokenize(log_content):
    words = log_content.split()
    words = [word for word in words if not re.search(r'\d', word)]
    return words


def vectorize(tokenized_logs):
    vectorizer = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False)
    return vectorizer.fit_transform(tokenized_logs)


def cluster(vectorized_logs, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(vectorized_logs)
    return kmeans.labels_


# 读取CSV文件
dataset = 'Spark'
df = pd.read_csv(
    f'dataset/{dataset}/{dataset}_2k.log_structured_corrected.csv')

# 选择某一列，例如'column_name'
column = df['Content']

# 将该列转换为列表
column_list = column.tolist()

tokenized_logs = [tokenize(content) for content in column_list]


labels = cluster(vectorize(tokenized_logs), 38)

C:\Users\xiaoyi\AppData\Roaming\Python\Python311\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\xiaoyi\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [47]:
num = 0
for i, l in enumerate(labels):
    if l == 37:  # 12, 14 , 17, 29是异常的模板
        print(column_list[i])
        num += 1
# 75 + 45 + 30
print(num)

Successfully started service 'org.apache.spark.network.netty.NettyBlockTransferService' on port 40984.
1


In [3]:

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd

# 读取CSV文件
dataset = 'Hadoop'
df = pd.read_csv(
    f'dataset/{dataset}/{dataset}_2k.log_structured_corrected.csv')

# 选择某一列，例如'column_name'
column = df['Content']

# 将该列转换为列表
column_list = column.tolist()

# 特征提取
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(column_list)

n = 115
# 聚类
kmeans = KMeans(n_clusters=n)  # n是模板的数量
kmeans.fit(X)

# 输出每个log message的分组结果
labels = kmeans.labels_

C:\Users\xiaoyi\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


显示每一批分组情况

In [11]:
num = 0
for i,l in enumerate(labels):
    if l == 8:  # 12, 14 , 17, 29是异常的模板
        print(column_list[i])
        num +=1
print(num)

task_1445144423722_0020_m_000000 Task Transitioned from NEW to SCHEDULED
task_1445144423722_0020_m_000002 Task Transitioned from NEW to SCHEDULED
task_1445144423722_0020_m_000003 Task Transitioned from NEW to SCHEDULED
task_1445144423722_0020_m_000005 Task Transitioned from NEW to SCHEDULED
task_1445144423722_0020_m_000006 Task Transitioned from NEW to SCHEDULED
task_1445144423722_0020_m_000008 Task Transitioned from NEW to SCHEDULED
task_1445144423722_0020_m_000009 Task Transitioned from NEW to SCHEDULED
task_1445144423722_0020_r_000000 Task Transitioned from NEW to SCHEDULED
8


通过三个参数选取最合适的n值

In [52]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import re

def tokenize(log_content):
    words = log_content.split()
    words = [word for word in words if not re.search(r'\d', word)]
    return words


def vectorize(tokenized_logs):
    vectorizer = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False)
    return vectorizer.fit_transform(tokenized_logs)


def cluster(vectorized_logs, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(vectorized_logs)
    # 计算聚类误差
    cluster_error = kmeans.inertia_
    # 计算轮廓系数
    silhouette_avg = silhouette_score(vectorized_logs, kmeans.labels_)
    # 计算Calinski-Harabasz指数
    calinski_harabasz_avg = calinski_harabasz_score(vectorized_logs.toarray(), kmeans.labels_)
    return kmeans.labels_, cluster_error, silhouette_avg, calinski_harabasz_avg

# 读取CSV文件
dataset = 'Hadoop'
df = pd.read_csv(f'dataset/{dataset}/{dataset}_2k.log_structured_corrected.csv')

# 选择某一列，例如'column_name'
column = df['Content']

# 将该列转换为列表
column_list = column.tolist()

tokenized_logs = [tokenize(content) for content in column_list]


vectorized_logs = vectorize(tokenized_logs)

range_n_clusters = range(50, 150)
# 聚类误差
errors = []

# 轮廓系数
silhouettes = []

# Calinski-Harabasz指数
calinski_harabasz_scores = []

for n_clusters in range_n_clusters:
    # 聚类
    labels, cluster_error, silhouette_avg, calinski_harabasz_avg = cluster(vectorized_logs, n_clusters)

    errors.append(cluster_error)
    silhouettes.append(silhouette_avg)
    calinski_harabasz_scores.append(calinski_harabasz_avg)

# 绘制聚类误差图
plt.figure()
plt.plot(range_n_clusters, errors, 'o-')
plt.xlabel('Number of clusters')
plt.ylabel('Cluster error')
plt.title('The Elbow Method')
plt.show()

# 绘制轮廓系数图
plt.figure()
plt.plot(range_n_clusters, silhouettes, 'o-')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')
plt.title('The Silhouette Method')
plt.show()

# 绘制Calinski-Harabasz指数图
plt.figure()
plt.plot(range_n_clusters, calinski_harabasz_scores, 'o-')
plt.xlabel('Number of clusters')
plt.ylabel('Calinski-Harabasz Index')
plt.title('The Calinski-Harabasz Method')
plt.show()

C:\Users\xiaoyi\AppData\Roaming\Python\Python311\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\xiaoyi\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\xiaoyi\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\xiaoyi\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_

KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_sparse'
Traceback (most recent call last):
  File "C:\Users\xiaoyi\AppData\Roaming\Python\Python311\site-packages\numpy\core\multiarray.py", line 346, in where
    @array_function_from_c_func_and_dispatcher(_multiarray_umath.where)
    
KeyboardInterrupt: 
C:\Users\xiaoyi\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:1152: ConvergenceWarning: Number of distinct clusters (115) found smaller than n_clusters (124). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
C:\Users\xiaoyi\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\xiaoyi\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:1152: ConvergenceWarning: 